# Libraries and imports

In [1]:
# NOTE: relaunch Jupyter if ipysigma widgets don't show
!pip install igraph leidenalg networkx pelote ipysigma fog

In [2]:
import math
import networkx as nx
import igraph as ig
import leidenalg as la
from copy import deepcopy
from pelote import read_graphology_json, triangular_strength, filter_edges
from pelote.graph import union_of_maximum_spanning_trees
from ipysigma import Sigma, SigmaGrid
from fog.metrics.utils import intersection_size

Sigma.set_defaults(max_categorical_colors=25)

# Misc helpers

In [3]:
def erase_node_positions(g: nx.Graph) -> None:
    for node, attr in g.nodes.data():
        try:
            del attr["x"]
            del attr["y"]
        except KeyError:
            pass

# Leiden helpers

In [4]:
def leiden_modularity(g: nx.Graph, weighted: bool = False):
    ig_g = ig.Graph.from_networkx(g)

    weights = None

    if weighted:
        weights = [float(w) for _, _, w in g.edges(data="weight")]
    
    partition = la.find_partition(ig_g, la.ModularityVertexPartition, weights=weights)
    return {n: m for n, m in zip(g, partition._membership)}

In [5]:
# Ref: https://github.com/graphology/graphology/pull/419
def leiden_modularity_with_ambiguities(g: nx.Graph, n: int, weighted: bool = False):
    assert n > 0
    
    partitions = [leiden_modularity(g, weighted=weighted) for _ in range(n)]

    node_registers = {}
    
    for node in g:
        node_registers[node] = {
            "entropyEdges": 0,
            "entropyNonEdges": 0,
            "nonNeighborsMet": 0,
        }

    def process_node_pair(n1, n2, is_edge: bool):
        if not is_edge:
            node_registers[n1]["nonNeighborsMet"] += 1
            node_registers[n2]["nonNeighborsMet"] += 1

        in_same_cluster_count = 0

        for p in partitions:
            if p[n1] == p[n2]:
                in_same_cluster_count += 1

        ratio = in_same_cluster_count / len(partitions)

        if in_same_cluster_count == 0 or in_same_cluster_count == len(partitions):
            return

        entropy = ((ratio - 1) * math.log(1 - ratio) - ratio * math.log(ratio)) / math.log(2)
        key = 'entropy' + ('Edges' if is_edge else 'NonEdges')

        node_registers[n1][key] += entropy
        node_registers[n2][key] += entropy

    # NOTE: we are not sampling pairs here, we do everything
    nodes = list(g)

    for i in range(len(nodes)):
        source = nodes[i]
        
        for j in range(i + 1, len(nodes)):
            target = nodes[j]

            has_edge = g.has_edge(source, target) or g.has_edge(target, source)
            process_node_pair(source, target, has_edge)

    ambiguities = {}

    for node, attr in node_registers.items():
        degree = g.degree(node)

        if attr['nonNeighborsMet'] != 0:
            ambiguity = (attr['entropyEdges'] + ((g.order() - degree) / attr['nonNeighborsMet']) * attr['entropyNonEdges']) / g.order()
        else:
            ambiguity = attr['entropyEdges'] / degree

        ambiguities[node] = ambiguity

    return partitions[0], ambiguities

# Checking the network

In [6]:
corpus = read_graphology_json('NETWORK_CorpusMedia_DEFACTO_medialab_SciencesPo_V1.json')
erase_node_positions(corpus)

In [7]:
Sigma(corpus)

Sigma(nx.DiGraph with 732 nodes and 27,556 edges)

# Computing weights for edges

In [8]:
# NOTE: I am duplicating the corpus to avoid messing with the Simmelian backbone later on
weighted_corpus = read_graphology_json('NETWORK_CorpusMedia_DEFACTO_medialab_SciencesPo_V1.json')
erase_node_positions(weighted_corpus)

node_count_sums = {}

for node in weighted_corpus:
    node_count_sums[node] = weighted_corpus.degree(node, weight="count")

# NOTE: each edge will get a weight that is a proportion of hyperlinks sent divided by the total count
# of hyperlinks sent by source entity
for source, target, attr in weighted_corpus.edges.data():
    attr["weight"] = attr["count"] / node_count_sums[source]

# NOTE: this will be used later on
normalized_weights = {((u, v) if u < v else (v, u)): 1 for u, v in weighted_corpus.edges()}

In [ ]:
weighted_corpus_communities, weighted_corpus_ambiguities = leiden_modularity_with_ambiguities(weighted_corpus, 50, weighted=True)

In [ ]:
Sigma(weighted_corpus, node_color=lambda n: -weighted_corpus_ambiguities[n], node_color_gradient=("yellow", "red"))

In [ ]:
SigmaGrid(
    weighted_corpus,
    default_node_border_color="white",
    node_size_range=(2, 10),
    edge_color="weight",
    edge_color_gradient=['#ddd', 'black'],
    edge_zindex="weight",
    views=[
        {"name": "Default colors"},
        {"name": "Leiden Weighted Modularity", "node_color": leiden_modularity(weighted_corpus, weighted=True)}
    ]
)

# Applying a Simmelian backbone

In [ ]:
def directed_simmelian_redudancies(graph: nx.DiGraph, m: int, weights=None, direction: str = 'in'):
    weights = triangular_strength(graph) if weights is None else weights

    neighbor_fn = graph.predecessors if direction == 'in' else graph.successors

    NN = {}

    for node in graph:
        weighted_neighbors = sorted(
            [(neighbor, weights[(node, neighbor) if node < neighbor else (neighbor, node)]) for neighbor in neighbor_fn(node)],
            key=lambda t: t[1],
            reverse=True
        )
        best_neighbors = weighted_neighbors[:m]

        # NOTE: ties must be included
        if len(best_neighbors) < len(weighted_neighbors):
            best_weight = best_neighbors[-1][1]
            i = len(best_neighbors)

            while i < len(weighted_neighbors):
                n = weighted_neighbors[i]
                
                if n[1] == best_weight:
                    best_neighbors.append(n)
                else:
                    break

                i += 1
        
        NN[node] = set(n[0] for n in best_neighbors)

    redundancies = {}

    for n1, n2 in graph.edges():
        if n1 > n2:
            (n1, n2) = (n2, n1)
        
        r = intersection_size(NN[n1], NN[n2])
        redundancies[(n1, n2)] = r

    return redundancies

In [ ]:
# NOTE: in the paper: m is the number of neighbors to consider when computing intersections
# and n is the redundancy threshold to keep the edge
def directed_simmelian_backbone(graph: nx.DiGraph, m: int, n: int, weights=None, direction: str = 'in', keep_connected: bool = False) -> nx.DiGraph:
    redundancies = directed_simmelian_redudancies(graph, m=m, weights=weights, direction=direction)

    umst = None

    if keep_connected:
        # NOTE: I am copying the graph to avoid pollutiong
        copy = deepcopy(graph)

        for source, target, attr in copy.edges.data():
            if source > target:
                source, target = target, source

            attr["weight"] = redundancies[source, target]

        umst = set((u, v) for u, v, _ in union_of_maximum_spanning_trees(copy))

    def edge_filter(source, target, attr) -> bool:
        if umst is not None and (source, target) in umst:
            return True
            
        if source > target:
            source, target = target, source

        return redundancies[source, target] > n

    return filter_edges(graph, edge_filter)

In [ ]:
# NOTE: increasing n means decreasing the number of edges in the result
# NOTE: increasing m means modifying the number of neighbors taken into account when computing redundancies
# NOTE: set keep_connected to True to avoid creating connected component by disconnecting nodes
# NOTE: set direction to 'in' or 'out' to consider the relevant edges
inbound_backbone = directed_simmelian_backbone(corpus, m=3, n=1, direction='in', keep_connected=True)

In [ ]:
outbound_backbone = directed_simmelian_backbone(corpus, m=3, n=1, direction='out', keep_connected=True)

In [ ]:
# NOTE: it is possible to use our weighted version to compute edge redundancy, but I find it
# weaker, because it is not tied to the graph's topology itself
inbound_weighted_backbone = directed_simmelian_backbone(
    weighted_corpus, m=3, n=1, keep_connected=False, direction='in',
    weights=normalized_weights
)

In [ ]:
# TODO: try jaccard or else, try both neighborhoods, try uncertainty louvain, lire le papier original du backbone

In [ ]:
inbound_backbone_communities, inbound_backbone_ambiguities = leiden_modularity_with_ambiguities(inbound_backbone, 50)

In [ ]:
Sigma(inbound_backbone, node_color=lambda n: -inbound_backbone_ambiguities[n], node_color_gradient='Plasma')

In [ ]:
SigmaGrid(
    inbound_backbone,
    default_node_border_color="black",
    node_size_range=(2, 10),
    views=[
        {"name": "Default colors"},
        {
            "name": "Leiden Weighted Modularity",
            "node_color": inbound_backbone_communities,
            "node_color_saturation": lambda n: -inbound_backbone_ambiguities[n],
            "node_color_saturation_scale": "log"
        }
    ]
)

In [ ]:
outbound_backbone_communities, outbound_backbone_ambiguities = leiden_modularity_with_ambiguities(outbound_backbone, 50)

In [ ]:
SigmaGrid(
    outbound_backbone,
    default_node_border_color="white",
    node_size_range=(2, 10),
    views=[
        {"name": "Default colors"},
        {"name": "Leiden Weighted Modularity", "node_color": outbound_backbone_communities, "node_color_saturation": lambda n: -outbound_backbone_ambiguities[n], "node_color_saturation_scale": "log"}
    ]
)